In [2]:
! pip install folium
! pip install geocoder

     |████████████████████████████████| 102kB 7.0MB/s ta 0:00:011
     |████████████████████████████████| 102kB 6.9MB/s ta 0:00:011


# Import Necessary libraries

In [3]:
import pandas as pd
import numpy as np
import folium 
import sklearn
import geocoder
from geopy.geocoders import Nominatim
import requests
import matplotlib.pyplot as plt
import matplotlib.colors as col
from matplotlib.colors import Colormap as cmap
from matplotlib.cm import ScalarMappable

# read the wikipedia list of mumbai areas and  locations into a DataFrame 

In [4]:
areas_df=pd.read_html('https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Mumbai')[0]

# Define your foursquare credentials

In [5]:
CLIENT_ID ='H1HEOTBF42KKGPEI3FBRTTWKOGG1FF4C13XNWR0KO1NA034D' # your Foursquare ID
CLIENT_SECRET = 'KMDVZ5X3LFVPWGC0MJJT0OQRZAOMUILUEGNZJUNRG5EZJL2V' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

categories=['4d4b7105d754a06374d81259']
LIMIT=100
RADIUS=500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('LIMIT:',LIMIT)
print('RADIUS:',RADIUS)


# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     ll[0], 
#     ll[1], 
#     radius, 
#     LIMIT)

Your credentails:
CLIENT_ID: H1HEOTBF42KKGPEI3FBRTTWKOGG1FF4C13XNWR0KO1NA034D
CLIENT_SECRET:KMDVZ5X3LFVPWGC0MJJT0OQRZAOMUILUEGNZJUNRG5EZJL2V
LIMIT: 100
RADIUS: 500


### Areas  and locations of the Mumbai City

In [ ]:
areas_df.head(12)

,Area,Location,Latitude,Longitude
0,Amboli,"Andheri,Western Suburbs",19.129300,72.843400
1,"Chakala, Andheri",Western Suburbs,19.111388,72.860833
2,D.N. Nagar,"Andheri,Western Suburbs",19.124085,72.831373
3,Four Bungalows,"Andheri,Western Suburbs",19.124714,72.827210
4,Lokhandwala,"Andheri,Western Suburbs",19.130815,72.829270
5,Marol,"Andheri,Western Suburbs",19.119219,72.882743
6,Sahar,"Andheri,Western Suburbs",19.098889,72.867222
7,Seven Bungalows,"Andheri,Western Suburbs",19.129052,72.817018
8,Versova,"Andheri,Western Suburbs",19.120000,72.820000
9,Mira Road,"Mira-Bhayandar,Western Suburbs",19.284167,72.871111


### Get latlngs from DF 

In [ ]:
latlngs=areas_df.apply(lambda x : (x[-2],x[-1]),axis=1)

#### This function gets an URL for every latitude and longitude 

In [ ]:
def get_url(latlng):
    ll=latlng
    url = 'https://api.foursquare.com/v2/venues/explore?categoryID={}&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    categories,CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ll[0], 
    ll[1], 
    RADIUS, 
    LIMIT)
    
    return url

In [ ]:
# map get_url function

urls=latlngs.map(get_url)

#### Mapping get requests to every URL

In [ ]:
resp=urls.map(requests.get)

#### Extract information from the json data if available else drop that url


In [ ]:

venues=resp.map(lambda x:x.json().get('response',0))
venues=venues.map(lambda x: x.get('groups',0) )

zeroind=venues[venues==0].index.tolist()

In [ ]:
venues=venues.drop(zeroind)

In [ ]:
venues=venues.map(lambda x: x[0])
venues=venues.map(lambda x:x.get('items',0))

#### Consider only items list which have atleast one venue or else drop that items list

In [ ]:
lens=venues.map(lambda x :len(x))
index=lens[lens==0].index
blank_items_index=index.tolist()

In [ ]:
venues=venues.drop(blank_items_index)

#### Latitudes and Longitudes /Locations which have atleast one venue

In [ ]:
# venues from foursquare
venues

#### Extract venue names,categories and latlngs for every venue with a for loop|

In [ ]:
rests_names_acc_to_area=[]
rests_locs_cc_to_area=[]
rests_cats_acc_to_area=[]

restaurants=[]

for venue in venues:
    
    restaurant_names=[]
    restaurant_locations=[]
    restaurant_categories=[]
    
    rests=[]
    
    for rest in venue:
        restaurant_names.append(rest['venue']['name'])
        restaurant_locations.append([rest['venue']['location']['lat'],rest['venue']['location']['lng']])
        restaurant_categories.append(rest['venue']['categories'][0]['name'])
        
        rests.append([rest['venue']['name'],[rest['venue']['location']['lat'],rest['venue']['location']['lng']],rest['venue']['categories'][0]['name']
        ])
        
    
    rests_names_acc_to_area.append(restaurant_names)
    rests_locs_cc_to_area.append(restaurant_locations)
    rests_cats_acc_to_area.append(restaurant_categories)
    
    restaurants.append(rests)

#### Create individual dataframes for above extracted properties

In [ ]:
rests_names_df=pd.DataFrame(rests_names_acc_to_area)
rests_locs_df=pd.DataFrame(rests_locs_cc_to_area)
rests_cats_df=pd.DataFrame(rests_cats_acc_to_area)

restaurants_df=pd.DataFrame(restaurants)

#### DataFrame showing every category that a specific Location has indexed same as the Original DataFrame

In [ ]:
rests_cats_df

In [ ]:
indus=rests_cats_df.index.tolist()

ppp=pd.merge(areas_df.iloc[indus,:],rests_cats_df,on=rests_cats_df.index).drop(['key_0'],axis=1)

#### Previous category DataFrame After merging with Original wikipedia Areas dataframe

In [ ]:
ppp

In [ ]:

columns=np.array(list(set([y for x in rests_cats_df.values.tolist() for y in x]))).reshape(-1,1)

df=pd.get_dummies(rests_cats_df.stack())

rests_cats_df=df.groupby(level=0).sum()

areas_with_categories_df=pd.concat([areas_df.iloc[rests_cats_df.index],rests_cats_df],axis=1)

In [ ]:
areas_with_categories_df=areas_with_categories_df.rename({'Café':'Cafe'},axis=1)    

#### Used Pd.get_dummies() on the categories to encode

In [ ]:
areas_with_categories_df

In [ ]:
f=areas_with_categories_df.iloc[:,4:].idxmax()

areas_rich_in=[]
indices=[x for x in zip(f.index,f.values)]

for x in indices :
    
    name,ind=x
    areas_rich_in.append([name,areas_with_categories_df.iloc[ind,:].Location])

## Category and the Location where it is famous for

In [ ]:
areas_rich_in_df=pd.DataFrame(areas_rich_in,columns=['Category','Location'])


areas_rich_in_df

## Locations and the categories they are rich in with count

Some Locations are rich in multiple categories e.g.:'Bandra Western Suburbs' is rich in 'Tea Room','Steak House' ..and many more

In [ ]:
areas_rich_in_df.groupby('Location').count().sort_values(by='Category',ascending=False).reset_index()

### Location with most categories 

In [ ]:
populous_place=areas_rich_in_df.groupby('Location').count().idxmax().values[0]

print(populous_place)


## South Mumbai and its Categories

In [ ]:

print(populous_place,'is rich in a total of',areas_rich_in_df.groupby('Location').count().loc['South Mumbai'].values[0],'venue categories')

ls=areas_rich_in_df[areas_rich_in_df.Location=='South Mumbai'].Category.values.tolist()

print(30*'----')

print('The Following are the categories:')

print(30*'----')
for l in ls:
    print(l+'s')

# By Far South Mumbai Looks Promising,but Lets look at other top Locations too

#### Function to give count of each category per location and threshold

In [ ]:
def category_count_per_area(df,location,threshold=4):
    series=df[df.Location==location].iloc[:,4:].sum()
    series=series[series>threshold]

    series_df=pd.DataFrame(series).reset_index().rename({'index':'Category',0:'Count'},axis=1)
    series_df['Location']=location
    return series_df



## Lets look at Locations which have a minimum category denisty of 5  ??<b>   
<i>E.g.:"If Bandra" has a minimum of 5 tea houses, add "Bandra".
Note:selection takes place irrespective of category<i>

In [ ]:
locs=[]

for loc in areas_with_categories_df.Location.unique():
    if len(category_count_per_area(df=areas_with_categories_df,location=loc,threshold=5))>0:
        locs.append([loc,len(category_count_per_area(df=areas_with_categories_df,location=loc,threshold=5))])

In [ ]:
locsdff=pd.DataFrame(locs,columns=['Location','Category_Count']).sort_values(by='Category_Count',ascending=False)

locsdff

### Filter Locations which do'nt have minimum varying categories (atleast 2 )

In [ ]:
filtered=locsdff[locsdff.iloc[:,1]>2].values.tolist()

filtered

<b> filtered Locations

In [ ]:
# After filtering we are left with 4 locations
filtered

In [ ]:
seri=[]

for loc in filtered:
    seri.append(category_count_per_area(df=areas_with_categories_df,location=loc[0],threshold=5))
    


## Now lets look at distribution for each Location

In [ ]:
# Function to give us a bar plot of each location
def plot_bar(series,cmap='plasma'):
    plt.figure(figsize=(9,4))

    my_cmap=plt.cm.get_cmap(cmap)


    colors=my_cmap([x/max(series.Count) for x in series.Count])


    bars=plt.bar(series.Category,series.Count,color=colors)

    sm = ScalarMappable(cmap=my_cmap, norm=plt.Normalize(0,max(series.Count)))
    sm.set_array([])

    cbar = plt.colorbar(sm)
    cbar.set_label('Frequency', rotation=270,labelpad=25)

    ax=plt.gca()
    ax.xaxis.set_ticklabels(series.Category,rotation=90)
    
    ax.yaxis.set_ticks([])
    
    plt.title(series.Location[0])
    
    return 


### Andheri Western Suburbs Distribution

In [ ]:

plot_bar(seri[1])

## Bandra,Western Suburbs Distribution

In [ ]:
plot_bar(seri[2])

## South Mumbai Distribution

In [ ]:
plot_bar(seri[0])

# Note : South Mumbai is both diverse and high in density of categories



In [ ]:
sortedcount=seri[0].sort_values(by='Count',ascending=False)
sortedcount[:]

# Of the above four graphs we can see that "SouthMumbai" and "Andheri western  Suburbs" looks promising

### Now with Folium Lets visualize clusters of "South Mumbai" and "Andheri western  Suburbs"

In [ ]:
locdf=areas_with_categories_df.loc[:,'Location']
latlngdf=pd.DataFrame(rests_locs_cc_to_area)



loc_with_latlngs_df=pd.merge(locdf,latlngdf,on=locdf.index)
loc_with_cats_df=pd.merge(locdf,pd.DataFrame(rests_cats_acc_to_area),on=locdf.index)

In [ ]:
from folium.plugins import MarkerCluster

sm=areas_with_categories_df[areas_with_categories_df.Location=='South Mumbai']

smlatlngs=sm.iloc[:,2:4].values[0]

south_mumbai_map=folium.Map(smlatlngs,zoom_start=12)

marker_cluster=MarkerCluster().add_to(south_mumbai_map)

In [ ]:
vals=loc_with_latlngs_df[loc_with_latlngs_df.Location=='South Mumbai'].iloc[:,2:].values
vals=vals[vals!=None]


alts=loc_with_cats_df[loc_with_cats_df.Location=='South Mumbai'].iloc[:,2:].values
alts=alts[alts!=None]

for x in range(len(vals)):
    folium.Marker(vals[x],popup=alts[x],tooltip='Category').add_to(marker_cluster)

# Marker Cluster Map of South Mumbai Distribution

We used Marker Cluster because it presents the map with clusters too

click on the clusters for detailed view

In [ ]:
south_mumbai_map

In [ ]:
from folium.plugins import MarkerCluster

bws=areas_with_categories_df[areas_with_categories_df.Location=='Andheri,Western Suburbs']
bwslatlngs=bws.iloc[:,2:4].values[0]

bws_map=folium.Map(bwslatlngs,zoom_start=12)

marker_cluster=MarkerCluster().add_to(bws_map)



In [ ]:
vs=loc_with_latlngs_df[loc_with_latlngs_df.Location=='Andheri,Western Suburbs'].iloc[:,2:].values
vs=vs[vs!=None]


ats=loc_with_cats_df[loc_with_cats_df.Location=='Andheri,Western Suburbs'].iloc[:,2:].values
ats=ats[ats!=None]

for x in range(len(vs)):
    folium.Marker(vs[x],popup=ats[x],tooltip='Category').add_to(marker_cluster)

## Bandra western Suburbs Map

In [ ]:
bws_map

# Looks like we can be inclined with South Mumbai 

## Find out what are the top 5 Categories from  South Mumbai 

In [ ]:
south_mumbai_df=areas_with_categories_df[areas_with_categories_df.Location=='South Mumbai']

seriessm=south_mumbai_df.iloc[:,4:].sum()
seriessm=seriessm[seriessm>5]

top5=seriessm.sort_values(ascending=False).index.tolist()

print('Top Venues with count more than 5:')

print('========================================')
for x in top5 :
    print(x)
    print(20*'=')


In [ ]:
south_mumbai_df=areas_with_categories_df[areas_with_categories_df.Location=='Andheri,Western Suburbs']

series=south_mumbai_df.iloc[:,4:].sum()
series=series[series>5]

top5=series.sort_values(ascending=False).index.tolist()

print('Venues with count more than 5:')

print('========================================')
for x in top5 :
    print(x)
    print(20*'=')


In [ ]:
plt.figure(figsize=(9,4))

my_cmap=plt.cm.get_cmap('plasma')

ss=seriessm[seriessm>14]

colors=my_cmap([x/max(ss.values) for x in ss.values])

bars=plt.bar(ss.index,ss.values,color=colors)

scm = ScalarMappable(cmap=my_cmap, norm=plt.Normalize(0,max(ss.values)))
scm.set_array([])

cbar = plt.colorbar(scm)
cbar.set_label('Frequency', rotation=270,labelpad=25)

ax=plt.gca()
ax.xaxis.set_ticklabels(ss.index,rotation=90)




# KMeans Clustering of South Mumbai

In [ ]:
from sklearn.cluster import KMeans

n_clusters=19

pred=KMeans(n_clusters).fit(sm.loc[:,'Latitude':'Longitude']).predict(sm.loc[:,'Latitude':'Longitude'])

sm.loc[:,'Cluster']=pred

sm

## Lets look at cluster 1

In [ ]:
sm_zero=sm[sm.Cluster==0].drop(['Location'],axis=1)

In [ ]:
def folium_map(df,cluster=None):
    ll=df.iloc[0,2][1]
    m=folium.Map([ll[0],ll[1]],zoom_start=11)
    
    if cluster:
    
        if cluster<n_clusters:
            Map=MarkerCluster().add_to(m)

            values=[y for x in df[df.Cluster==cluster].iloc[:,2:].values for y in x if y not in [None,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] and len(y)==3]
            vals=pd.DataFrame(values,columns=['name','cords','category'])
            vals.apply(lambda x : folium.CircleMarker([x.cords[0],x.cords[1]],popup=x.values[0],tooltip='venue name',fill_opacity=0.9,radius=8,color='red',fill_color='orange').add_to(Map),axis=1)
        else:
            raise TypeError('Not valid cluster number')
        
    elif cluster==None:
        m=folium.Map([ll[0],ll[1]],zoom_start=14)
       
        values=[y for x in df.iloc[:,2:].values for y in x if y not in [None,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] and len(y)==3]
        vals=pd.DataFrame(values,columns=['name','cords','category'])
        vals.apply(lambda x : folium.CircleMarker([x.cords[0],x.cords[1]],popup=x.values[0],radius=0.5,tooltip='venue name',color='red',fill_color='k').add_to(m),axis=1)
    return m      

In [ ]:
with_latlngs_df=pd.merge(locdf,pd.DataFrame(restaurants),on=locdf.index)

with_latlngs_df=with_latlngs_df[with_latlngs_df.Location=='South Mumbai']

with_latlngs_df.loc[:,'Cluster']=pred



 # Cluster one Folium Map

In [ ]:
folium_map(with_latlngs_df,0)

# Second Cluster :

In [ ]:
folium_map(with_latlngs_df,1)

# Third Cluster

In [ ]:
folium_map(with_latlngs_df,2)

# Fourth Cluster and venues

In [ ]:
folium_map(with_latlngs_df,3)

# Fifth Cluster and venues

In [ ]:
folium_map(with_latlngs_df,4)

# Sixth cluster

In [ ]:
folium_map(with_latlngs_df,5)

# Seventh cluster

In [ ]:
folium_map(with_latlngs_df,6)

# Eighth cluster

In [ ]:
folium_map(with_latlngs_df,7)

# Ninth Cluster

In [ ]:
folium_map(with_latlngs_df,8)

# Tenth Cluster

In [ ]:
folium_map(with_latlngs_df,9)

# Eleventh Cluster

In [ ]:
folium_map(with_latlngs_df,10)

# Twelveth Cluster

In [ ]:
folium_map(with_latlngs_df,11)

# Thirteenth Cluster

In [ ]:
folium_map(with_latlngs_df,12)

# Fourteenth Cluster

In [ ]:
folium_map(with_latlngs_df,13)

# Fifteenth Cluster

In [ ]:
folium_map(with_latlngs_df,14)

# Sixteenth Cluster

In [ ]:
folium_map(with_latlngs_df,15)

# Seventeenth Cluster

In [ ]:
folium_map(with_latlngs_df,16)

# Eighteenth Cluster

In [ ]:
folium_map(with_latlngs_df,17)

# Nineteenth Cluster

In [ ]:
folium_map(with_latlngs_df,18)

In [ ]:
folium_map(with_latlngs_df)

In [ ]:
# pd.merge(areas_df,rests_cats_df,on=areas_df.index)

# areas_df.shape

rests_cats_df